# Content-based Filtering

The content-based filtering recommendation algorithm suggests items similar to ones the user has liked before by analyzing the features of the items and the user’s preferences. It compares the content (e.g., product descriptions, keywords, tags) of items with a user’s historical interactions (such as movies they have watched or articles they have read) and suggests items with similar attributes.

Imagine a user watching a sci-fi movie, and the system extracts features like “genre: sci-fi,” “director: Christopher Nolan,” and “themes: space exploration.” The system would then recommend other movies sharing these features, e.g., another sci-fi film with similar themes and directors.

In this notebook, we would take a very simple and straight-forward approach to implement Content-based Filtering. We make recommendations by finding the most similar items to the user's last interacted item, where similarity is defined based on the textual description of the items. The score is the similarity between the target item and the neighbors.

Specifically, we will:
- Select a few description fields from the item metadata
- Transform the features into numerical format, e.g. TFIDF for text data, Multi-Hot Encoders for categorical features
- Concat and normalize the transformation output
- Use cosine similarity to look up neighbor items and use it as the prediction score

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import dill
import mlflow
import numpy as np  # required for the scikit-learn pipeline to work
import pandas as pd
import torch
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from sklearn.preprocessing import MinMaxScaler

load_dotenv()

sys.path.insert(0, "..")


from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.model import ContentBased
from src.train_utils import map_indice

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "005-content-based"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-24 07:55:47.759 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 005-content-based...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "005-content-based",
  "notebook_persist_dp": "/home/jupyter/frostmourne/reco-algo/notebooks/data/005-content-based",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(train_item_features, device):
    model = ContentBased(item_features=train_item_features, device=device)
    return model

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
# device = 'cpu'
logger.info(f"Using {device} device")

2024-09-24 07:55:47.924 | INFO     | __main__:<module>:7 - Using cuda device


In [6]:
with open("../data/item_metadata_pipeline.dill", "rb") as f:
    item_metadata_pipeline = dill.load(f)

# Test implementation

In [7]:
# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
ratings = [1, 4, 5, 3, 2]
timestamps = [0, 1, 2, 3, 4]
main_category = [
    "All Electronics",
    "Video Games",
    "All Electronics",
    "Video Games",
    "Unknown",
]
title = ["All Electronics", "Video Games", "All Electronics", "Video Games", "Unknown"]
description = [[], [], ["Video games blah blah"], [], ["blah blah"]]
categories = [[], ["Headsets"], ["Video Games"], [], ["blah blah"]]
price = ["from 14.99", "14.99", "price: 9.99", "20 dollars", "None"]

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.rating_col: ratings,
        args.timestamp_col: timestamps,
        "main_category": main_category,
        "title": title,
        "description": description,
        "categories": categories,
        "price": price,
    }
)
# Drop duplicated item features so that the ContentBased model will fit correctly in terms of index mapping
fit_df = train_df.drop_duplicates(subset=["item_indice"])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]
val_timestamps = [5, 6, 7]
val_main_category = ["All Electronics", "Video Games", "All Electronics"]
val_title = ["All Electronics", "Video Games", "All Electronics"]
val_description = [["Video games blah blah"], [], ["Video games blah blah"]]
val_categories = [["Video Games"], ["Headsets"], ["Video Games"]]
val_price = ["price: 9.99", "14.99", "price: 9.99"]

val_df = pd.DataFrame(
    {
        "user_indice": val_user_indices,
        "item_indice": val_item_indices,
        args.rating_col: val_ratings,
        args.timestamp_col: val_timestamps,
        "main_category": val_main_category,
        "title": val_title,
        "description": val_description,
        "categories": val_categories,
        "price": val_price,
    }
)
val_item_features = item_metadata_pipeline.transform(val_df).astype(np.float32)

In [8]:
n_users = len(set(user_indices))
n_items = len(set(item_indices))

model = init_model(train_item_features, device)

items1 = [1, 2]
items2 = [0, 3]
predictions = model.predict(items1, items2)
print(predictions)

print("\n\n")

users = [0, 1]
anchor_items = [2, 3]
recommendations = model.recommend(users, anchor_items, k=args.top_K)
print(recommendations)

tensor([0.1171, 0.0386], device='cuda:0')





Generating Recommendations:   0%|          | 0/2 [00:00<?, ?it/s]

{'user_indice': [0, 0, 0, 0, 1, 1, 1, 1], 'recommendation': [0, 4, 1, 3, 1, 4, 0, 2], 'score': [0.6484283804893494, 0.1365058422088623, 0.03903510421514511, 0.03860381618142128, 0.9609925150871277, 0.3168308436870575, 0.16951429843902588, 0.03860381618142128]}


# Prep data

In [9]:
def get_last_item(df, item_sequence_col="item_sequence"):
    return df.assign(
        last_item_indice=lambda df: df[item_sequence_col].apply(lambda s: s[-1])
    )

In [10]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [11]:
train_df = train_df.pipe(get_last_item)
val_df = val_df.pipe(get_last_item)
full_df = pd.concat([train_df, val_df], axis=0)
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-24 07:55:54.363 | INFO     | __main__:<module>:9 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [12]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)
full_df = full_df.pipe(map_indice, idm, args.user_col, args.item_col)

In [13]:
train_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
0,AGHM4EAPVTF3AT2KYZXOK6HKNOXA,B0001ZZNME,0.0,1407432084000,13171,2286,Video Games,The Legend of Zelda - Classic NES Series,"[From the Manufacturer, Embark on a quest to f...","[Video Games, Legacy Systems, Nintendo Systems...",52.49,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....",26.0
1,AHA4MXZFGXAJ5FG734SZJOSR2U3Q,B00TEDK8FQ,0.0,1471888794000,7740,2636,Video Games,Ortz PS4 Vertical Stand with Cooling Fan [Keep...,[],"[Video Games, PlayStation 4, Accessories, Case...",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 331...",2649.0
2,AGHEHQXPA2XCTMDO2LIXNU3SEOBA,B00004U5VI,0.0,1489203706000,9408,306,Video Games,Gauntlet: Dark Legacy,[],"[Video Games, Legacy Systems, PlayStation Syst...",54.6,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 327...",1230.0
3,AE6DGY6HUKOK7DFMO27RZBQYYLCA,B01LP5V5CU,0.0,1321017752000,12163,1237,Video Games,Controller Gear Officially Licensed Gears of W...,[Officially Licensed and Exclusively Designed ...,"[Video Games, Xbox One, Accessories, Controlle...",None,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 2137.0, 4...",4199.0
4,AFAUFOADANHRMH3GR2FA73A6NHIA,B08VFQ3XJX,5.0,1408655680000,13360,1830,Video Games,Final Fantasy X,"[Product Description, Final Fantasy X finally ...","[Video Games, Legacy Systems, PlayStation Syst...",20.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 2826.0, 2...",1579.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340085,AGA2NNJEVTFU5RPOJEFUEE2D7K2Q,B00W435BL4,0.0,1531255931610,9421,686,Video Games,Madden NFL 16 - PlayStation 3,[Be The Playmaker with Madden NFL 16],"[Video Games, Legacy Systems, PlayStation Syst...",35.03,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 422...",3762.0
340086,AHF35NTC74W2QLLY442BYFZV2NZQ,B00BN5T30E,0.0,1508829250207,5469,1145,Video Games,Assassin's Creed IV Black Flag - PC,"[From the Manufacturer, Assassin's Creed, ®, I...","[Video Games, PC, Games]",43.89,"[3745.0, 3352.0, 3441.0, 3951.0, 3851.0, 1451....",1364.0
340087,AFEB7PR5ZKA7OLF2YBVAZJTS6VFQ,B07D36WX84,0.0,1593471355138,18180,2056,All Electronics,FastSnail Controller Charger Compatible with N...,[],"[Video Games, Legacy Systems, Nintendo Systems...",19.99,"[1472.0, 3445.0, 4295.0, 2897.0, 1058.0, 610.0...",2045.0
340088,AEFJKBRX2TPPMMJ53DKHANKRYDXQ,B08N6NMGNB,5.0,1510350967758,1452,6,Video Games,Thrustmaster T300 RS - Gran Turismo Edition Ra...,[Works with PS5 games (PS5 games compatibility...,"[Video Games, Legacy Systems, PlayStation Syst...",449.0,"[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, 316...",4555.0


# Fit

In [14]:
fit_df = train_df.drop_duplicates(subset=[args.item_col])
train_item_features = item_metadata_pipeline.transform(fit_df).astype(np.float32)
model = init_model(train_item_features, device)

# Predict

In [15]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice
698,AEMJNEOYWDSJ2KUUTWSSTGADKBJQ,B08N6NT1BW,3.0,1633505277199,1931,2589,Video Games,Thrustmaster TMX Racing Wheel with force feedb...,[Official Racing Simulator for Xbox One and Wi...,"[Video Games, PC, Accessories, Controllers, Ra...",199.0,"[-1, -1, -1, -1, -1, 2331, 4538, 1254, 390, 2824]",2824
1706,AEMJNEOYWDSJ2KUUTWSSTGADKBJQ,B006VR663G,0.0,1633505277199,1931,1261,Video Games,The Amazing Spider-Man - Nintendo Wii U,[Harness Spider-Man's powers with Manhattan as...,"[Video Games, Legacy Systems, Nintendo Systems...",79.17,"[-1, -1, -1, -1, -1, 2331, 4538, 1254, 390, 2824]",2824


In [16]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
anchor_item_indice = test_df["last_item_indice"].values[0]
item_indice = idm.get_item_index(item_id)
model.predict([item_indice], [anchor_item_indice])

2024-09-24 07:55:59.375 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AEMJNEOYWDSJ2KUUTWSSTGADKBJQ and parent_asin = B08N6NT1BW


tensor([-0.0029], device='cuda:0')

# Recommend

In [17]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_user_indices = val_df["user_indice"].values

In [18]:
recommendations = model.recommend(val_user_indices, val_anchor_item_indices, args.top_K)

Generating Recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

# Evaluate

## Ranking metrics

In [19]:
recommendations_df = pd.DataFrame(recommendations).pipe(
    create_rec_df, idm, args.user_col, args.item_col
)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,12853,4446,0.552764,1.0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B002I0K3PM
1,12853,4263,0.489624,3.0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B002I08RR8
2,12853,3570,0.451447,5.0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B00005BZCH
3,12853,400,0.403060,7.0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B07BF2422D
4,12853,4006,0.335957,9.0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B0056YAIZW
...,...,...,...,...,...,...
189795,14633,3462,0.056625,192.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001EYUXIK
189796,14633,1062,0.056444,194.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001ELJFP0
189797,14633,484,0.055910,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B07DJX3Y47
189798,14633,4475,0.055672,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B0050SW4OC


In [20]:
label_df = create_label_df(
    val_df,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
    timestamp_col=args.timestamp_col,
)
label_df

,user_id,parent_asin,rating,rating_rank
1727,AEOY2365QPPEVDTOXL6N7ZA4NSAA,B00PDRZG9U,5.0,1.0
451,AFGHX4VLP6P5XORLDJX3LZKUAAZA,B00Z9TJBUW,5.0,1.0
204,AFCH2PDOFM2S3622QFV6PHCHGMCA,B00KSQHX1K,5.0,1.0
1344,AEURBISVS35ALE7YQLR5L4K7AHCA,B07QQ8N7LL,1.0,1.0
334,AEMA3SW3WPNLEH3IACW23K2ZSUFA,B09JDLC31H,4.0,1.0
...,...,...,...,...
1332,AFB6FYPPCN33UMUU5536IHXNOHCQ,B002I0K3CK,0.0,18.0
910,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07BLRF329,0.0,18.0
695,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B001ELJDWA,0.0,18.0
1177,AFB6FYPPCN33UMUU5536IHXNOHCQ,B003S6N7OO,0.0,19.0


In [21]:
eval_df = merge_recs_with_target(
    recommendations_df,
    label_df,
    k=args.top_K,
    user_col=args.user_col,
    item_col=args.item_col,
    rating_col=args.rating_col,
)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
22,9912.0,391.0,0.478342,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B000B6MLU0,0,NaN
23,9912.0,391.0,0.478342,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B000B6MLU0,0,NaN
146,9912.0,1804.0,0.357686,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B072HGLS26,0,NaN
147,9912.0,1804.0,0.357686,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B072HGLS26,0,NaN
14,9912.0,2166.0,0.340666,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B00006FDLP,0,NaN
...,...,...,...,...,...,...,...,...
191551,2049.0,1964.0,0.072340,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00KSY5486,0,NaN
191490,2049.0,430.0,0.071783,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B002GN8HU4,0,NaN
191491,2049.0,430.0,0.071783,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B002GN8HU4,0,NaN
191482,2049.0,2867.0,0.071105,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B001EYUX4O,0,NaN


In [22]:
ranking_report = log_ranking_metrics(args, eval_df)

/home/jupyter/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [23]:
val_anchor_item_indices = val_df["last_item_indice"].values
val_item_indices = val_df["item_indice"].values

In [24]:
classifications = (
    model.predict(val_item_indices, val_anchor_item_indices)
    .cpu()
    .detach()
    .numpy()
    .reshape(-1, 1)
)
classifications = MinMaxScaler(feature_range=(0, 1)).fit_transform(classifications)

In [25]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,last_item_indice,classification_proba,label
0,AGXTHABHPC3XO4VAMCFM2TQR3GFQ,B00TEDK8FQ,0.0,1643101921864,12853,2636,Video Games,Ortz PS4 Vertical Stand with Cooling Fan [Keep...,[],"[Video Games, PlayStation 4, Accessories, Case...",None,"[111, 3920, 3879, 3261, 3402, 1230, 2239, 3974...",3479,0.121441,0
1,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B07BMRGKX2,0.0,1653590691326,141,253,Video Games,Agony - PlayStation 4,"[Agony is a first-person, survival horror game...","[Video Games, PlayStation 4, Games]",28.0,"[99, 4672, 4434, 1551, 1561, 2497, 3615, 3196,...",3998,0.115742,0
2,AEXFEQ7QOP6EHDEZ3K6NN27MQ7KA,B0774N9JKW,0.0,1651718479413,13969,1217,Video Games,Sword Art Online: Hollow Realization - PlaySta...,"[""Link start"" into SWORD ART ONLINE -Hollow Re...","[Video Games, PlayStation 4, Games]",18.11,"[1016, 3999, 2944, 742, 3161, 3580, 2267, 3623...",3015,0.116765,0
3,AGDAPPCYV472FOUKDGAHZRW766GA,B07B416X7V,0.0,1649310595659,18594,3519,Video Games,Burnout Paradise Remastered - Xbox One [Digita...,[Make action your middle name as you rule the ...,"[Video Games, Xbox One, Games]",None,"[3097, 2497, 3203, 3937, 3803, 2323, 2310, 751...",525,0.085149,0
4,AG2KBJG5DMEIISPJVF3OVMRB4ALA,B001D8Q5MA,0.0,1636861380056,5042,3122,Video Games,Grand Theft Auto IV [Online Game Code],"[From the Manufacturer, What does the American...","[Video Games, PC]",None,"[-1, -1, -1, -1, -1, 3602, 22, 1193, 2486, 1293]",1293,0.148000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AGVBR47VU2BE4EVWFAXOO26SOWSA,B0C39GFK7P,1.0,1647361584062,388,4128,Computers,"Logitech G640 Large Cloth Gaming Mouse Pad, Op...",[The cloth surface of G640 provides ideal surf...,"[Video Games, PC, Accessories, Gaming Mice]",29.99,"[1938, 2662, 262, 3903, 3610, 1896, 1372, 4160...",4647,0.173676,1
1894,AFUWPAK6VCGEL2OVIL2YGZNFQJZQ,B08N6NCR3Q,4.0,1642699950266,4205,3269,Video Games,Thrustmaster T 16000M SPACE SIM DUO STICK (PC),[The THRUSTMASTER T.16000M FCS Space Sim Duo c...,"[Video Games, PC, Accessories, Controllers, Fl...",119.51,"[-1, -1, -1, -1, 1058, 3558, 377, 1187, 2169, ...",4138,0.140587,1
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,20004,4190,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 832, 3126, 1490, 4335, 2035, 1270, 605, 3...",1404,0.186799,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,4984,2257,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4269, 4366, 396, 3060, 464]",464,0.138953,1


In [26]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

# Clean up

In [27]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/24 07:56:04 INFO mlflow.tracking._tracking_service.client: 🏃 View run 005-content-based at: http://localhost:5003/#/experiments/2/runs/0df00c5f5cb94c169192ab955654fdcf.
2024/09/24 07:56:04 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/2.
